## Import all the necesary libraries

Source: https://github.com/akshat57/Blind-Descent/blob/main/Blind_Descent-1-CNN.ipynb

In [1]:
import numpy as np
import torch
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils import data
from torchvision import transforms
from torchvision.datasets import MNIST

import matplotlib.pyplot as plt
import time

from sklearn.metrics import confusion_matrix, classification_report

cuda = torch.cuda.is_available()
cuda

True

## Download the MNIST Train and Test set & Compute basic data statistics
It is always important to know the shape of the data, as well as the min/max and mean/variance

In [2]:
train = MNIST('./data', train=True, download=True, transform=transforms.ToTensor())
test = MNIST('./data', train=False, download=True, transform=transforms.ToTensor())
train_data = train.train_data
train_data = train.transform(train_data.numpy())

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


## Dataloader

In [3]:
class MyDataset(data.Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)

    def __getitem__(self,index):
        X = self.X[index].reshape(1,28,28).float()
        Y = self.Y[index].long()
        return X,Y

Using the torch.utils.data DataLoader, we shuffle the data and set the batch size to be 256

In [4]:
num_workers = 8 if cuda else 0 
batch_size = 64
    
# Training
train_dataset = MyDataset(train.train_data, train.train_labels)

train_loader_args = dict(shuffle=True, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=batch_size)
train_loader = data.DataLoader(train_dataset, **train_loader_args)

# Testing
test_dataset = MyDataset(test.test_data, test.test_labels)

test_loader_args = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
test_loader = data.DataLoader(test_dataset, **test_loader_args)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## Define our Neural Network Model 
We define our model using the torch.nn.Module class

In [5]:
class MyCNN_Model(nn.Module):
    def __init__(self):
        super(MyCNN_Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding = 1)
        self.pool = nn.MaxPool2d( kernel_size = 3, stride = 2, padding = 1)
        
        self.fc1 = nn.Linear(16 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(-1, 16 * 14 * 14)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Create the model and define the Loss and Optimizer
We create an instance of our Simple_MLP model by passing it a list of layer sizes. The input layer contains 28*28 = 784 neurons. We define a single hidden layer of size 256, and our output will be the class probabilities for each of the 10 digits (0-9). 

Since this is a classification task, we will use Cross Entropy Loss. We define our criterion using the torch.nn.CrossEntropyLoss.

In order to train our network, we use the torch.optim.SGD optimizer.

In [6]:
model = MyCNN_Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

MyCNN_Model(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [7]:
def train_epoch(model, train_loader, criterion, optimizer, epoch, lr):
    model.train()

    running_loss = 0.0
    predictions = []
    ground_truth = []
    loss_counter = 1000
    loss_den = 0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
            data = data.to(device)
            target = target.to(device)
        
            #previous model
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions = target.size(0)
            correct_predictions = (predicted == target).sum().item()
            acc = (correct_predictions/total_predictions)*100.0
            
            loss = criterion(outputs, target)
            
            #new model
            model_new = MyCNN_Model() 
            model_new.conv1.weight = nn.Parameter(torch.from_numpy(np.random.normal(model.conv1.weight.detach().cpu().numpy(), scale = lr)).float())
            model_new.fc1.weight = nn.Parameter(torch.from_numpy(np.random.normal(model.fc1.weight.detach().cpu().numpy(), scale = lr)).float())
            model_new.fc2.weight = nn.Parameter(torch.from_numpy(np.random.normal(model.fc2.weight.detach().cpu().numpy(), scale = lr)).float())
            model_new.to(device)
            
            outputs = model_new(data)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions = target.size(0)
            correct_predictions = (predicted == target).sum().item()
            acc_new = (correct_predictions/total_predictions)*100.0
            
            loss_new = criterion(outputs, target)
        
            if loss_new.item() < loss.item():
                loss_den += 1
                loss_counter = loss_new.item()
                running_loss += loss_new.item()
                model = model_new
                #calculuating confusion matrix
                predictions += list(predicted.detach().cpu().numpy())
                ground_truth += list(target.detach().cpu().numpy())
        
        
    
    end_time = time.time()
    running_loss /= loss_den
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    #write_confusion_matrix('Training', ground_truth, predictions)

    
    return running_loss, model

## Create a function that will evaluate our network's performance on the test set

In [8]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        
        predictions = []
        ground_truth = []

        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()
            
            #calculuating confusion matrix
            predictions += list(predicted.detach().cpu().numpy())
            ground_truth += list(target.detach().cpu().numpy())
    

    
        #write_confusion_matrix('Testing', ground_truth, predictions)
        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Testing Loss: ', running_loss)
        print('Testing Accuracy: ', acc, '%')
        return running_loss, acc


## Train the model for N epochs
We call our training and testing functions in a loop, while keeping track of the losses and accuracy. 

In [ ]:
n_epochs = 40
Train_loss = []
Test_loss = []
Test_acc = []
lr = 0.001

for i in range(n_epochs):
    train_loss, model = train_epoch(model, train_loader, criterion, optimizer, i, lr)
    test_loss, test_acc = test_model(model, test_loader, criterion)
    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)
    print('='*20)